In [1]:
import spacy
from spacy.training import Example, offsets_to_biluo_tags
from spacy.training.corpus import Corpus
from spacy.training.loop import train
from spacy.util import load_config

from spacy import displacy

from sklearn.model_selection import train_test_split
import json

from spacy.scorer import Scorer


In [2]:
# Função para converter seu JSON anotado
def convert_json_to_spacy_format(json_data):
    spacy_format_data = []

    for entry in json_data:
        text = entry['text']
        entities = entry['label']  # Pegando o campo "label" que contém as entidades
        spacy_entities = []
        
        # Convertendo cada entidade para o formato que o spaCy espera
        for entity in entities:
            start, end, label = entity
            spacy_entities.append((start, end, label))
        
        # Adicionando ao formato do spaCy: (texto, {"entities": [(início, fim, rótulo)]})
        spacy_format_data.append((text, {"entities": spacy_entities}))
    
    return spacy_format_data

In [ ]:
with open('data\\corpus_annotated\\henrique.jsonl', 'r', encoding='utf-8') as f:
    json_data = [json.loads(line) for line in f]

for entry in json_data:
    print(entry)

In [4]:
train_data, test_data = train_test_split(json_data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.125, random_state=42)

In [ ]:
print(f'Tamanho do treino: {len(train_data)}')
print(f'Tamanho da validação: {len(val_data)}')
print(f'Tamanho do teste: {len(test_data)}')

In [6]:
# Convertendo o JSON para Spacy
train_data_spacy = convert_json_to_spacy_format(train_data)
val_data_spacy = convert_json_to_spacy_format(val_data)
test_data_spacy = convert_json_to_spacy_format(test_data)

In [7]:
nlp = spacy.blank("pt")  # Modelo vazio em português
ner = nlp.add_pipe("ner")

In [8]:
# Adicionar os rótulos das entidades
for _, annotations in train_data_spacy:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

##### Treinamento do modelo

In [ ]:
optimizer = nlp.begin_training()

for itn in range(10):  # Número de épocas
    print(f"Iteração {itn + 1}")
    for text, annotations in train_data_spacy:
        # Atualizando o modelo com os textos e entidades
        doc = nlp.make_doc(text)
        example = spacy.training.Example.from_dict(doc, annotations)
        nlp.update([example], sgd=optimizer)

##### Avaliação do modelo

In [ ]:
examples = []
for item in test_data:
    doc = nlp.make_doc(item["text"])
    entities = item["label"]
    example = Example.from_dict(doc, {"entities": entities})
    examples.append(example)

In [11]:
# Avaliação do modelo
scores = nlp.evaluate(examples)

In [ ]:
# Acessar a precisão geral
print("Precisão geral:", scores["ents_p"])
print("Recall:", scores["ents_r"])
print("F1 Score:", scores["ents_f"])

# Acessar métricas específicas para cada tipo de entidade
for ent_type, metrics in scores["ents_per_type"].items():
    print(f"\nEntidade: {ent_type}")
    print(f" - Precision: {metrics['p']}")
    print(f" - Recall: {metrics['r']}")
    print(f" - F1 Score: {metrics['f']}")


In [ ]:
text = "O Inter perdeu de 2 a 0 na Copa Libertadores para o Grêmio. O treinador foi o Celso Roth. No estádio Beira Rio."
doc = nlp(text)
displacy.render(doc, style="ent", jupyter=True)